In [3]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

In [4]:
def convert_pdf_to_txt(file):
    
    rsrcmgr = PDFResourceManager()
    
    retstr = StringIO()
    
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams = laparams)
    fp = open(file, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos = set()
    
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password = password, caching=caching, check_extractable = True):
        interpreter.process_page(page)
    
    text = retstr.getvalue()
    fp.close()
    device.close()
    retstr.close()
    return text

In [5]:
def bok_crawl():
    final_list = []
    for i in range(1,23):
        
        print('현재 {} 페이지를 진행 중입니다'.format(i))

        url = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761'
        params = {
        'pageIndex' : i
        }
        
        resp = requests.get(url, params=params)
        soup = BeautifulSoup(resp.content)
        
        ul = soup.select('div.bdLine ul div.row')
        
        result_dict = {}
        result = []
        
        for x in range(len(ul)):
            if x%2 == 0:
                link_li = ul[x].find('div', class_='fileGoupBox').find('ul').find_all('li')
                for link in link_li:
                    if link.find('a').text.strip()[-3:]=='pdf':
                        pdf_url = link.find('a').get('href')
                        full_url = 'http://www.bok.or.kr' + pdf_url
                        title = link.find('a').text.strip()

                        resp2 = requests.get(full_url)

                        with open('./MPC/'+ title, 'wb') as f:
                            f.write(resp2.content)

                        text = convert_pdf_to_txt(title)
                        text = re.sub('\n|\t|\r|\x0c', '', text)

                        result_dict = {
                            'title':ul[x].find('span', class_='titlesub').text,
                            'date':ul[x].find('span', class_='date').text[3:],
                            'contents':text
                        }
                        
                        result.append(result_dict)
                        
        final_list.extend(result)
    
    df = pd.DataFrame(final_list)
    df.to_csv('mpc_minute.csv', encoding='utf-8-sig')    
    
    return df



SyntaxError: invalid syntax (<ipython-input-5-e98ba2621b6b>, line 31)

In [7]:
bok_crawl()

현재 1 페이지를 진행 중입니다
현재 2 페이지를 진행 중입니다
현재 3 페이지를 진행 중입니다
현재 4 페이지를 진행 중입니다
현재 5 페이지를 진행 중입니다
현재 6 페이지를 진행 중입니다
현재 7 페이지를 진행 중입니다
현재 8 페이지를 진행 중입니다
현재 9 페이지를 진행 중입니다
현재 10 페이지를 진행 중입니다
현재 11 페이지를 진행 중입니다
현재 12 페이지를 진행 중입니다
현재 13 페이지를 진행 중입니다
현재 14 페이지를 진행 중입니다
현재 15 페이지를 진행 중입니다
현재 16 페이지를 진행 중입니다
현재 17 페이지를 진행 중입니다
현재 18 페이지를 진행 중입니다
현재 19 페이지를 진행 중입니다
현재 20 페이지를 진행 중입니다
현재 21 페이지를 진행 중입니다
현재 22 페이지를 진행 중입니다


,title,date,contents
0,금융통화위원회 의사록(2020년도 제17차)(2020.7.30.),2020.08.18,2020년도 제17차금융통화위원회(정기) 의사록한 국 은 행1. 일2. 장자소3. ...
1,금융통화위원회 의사록(2020년도 제16차)(2020.7.17.),2020.08.04,2020년도 제16차금융통화위원회(임시) 의사록한 국 은 행1. 일2. 장자소3. ...
2,금융통화위원회 의사록(2020년도 제15차)(2020.7.16.),2020.08.04,2020년도 제15차금융통화위원회(정기) 의사록한 국 은 행1. 일자2020년 7월...
3,금융통화위원회 의사록(2020년도 제14차)(2020.6.24.),2020.07.10,2020년도 제14차금융통화위원회(정기) 의사록한 국 은 행1. 일자2020년 6월...
4,금융통화위원회 의사록(2020년도 제13차)(2020.6.11.),2020.06.30,2020년도 제13차금융통화위원회(정기) 의사록한 국 은 행1. 일2. 장자소3. ...
...,...,...,...
215,금융통화위원회 의사록(2009년도 제26차)(2009.12.10),2010.01.26,금융통화위원회 의사록2009년도 제 차 회의261.일자2009년 월 일 목)1210...
216,금융통화위원회 의사록(2009년도 제25차)(2009.11.26),2010.01.12,금융통화위원회 의사록2009년도 제 차 회의251.일자2009년 월 일 목)1126...
217,금융통화위원회 의사록(2009년도 제24차)(2009.11.12),2009.12.29,금융통화위원회 의사록2009년도 제 차 회의241.일자2009년 월 일 목)1112...
218,금융통화위원회 의사록(2009년도 제22차)(2009.10.9),2009.11.24,금융통화위원회 의사록2009년도 제 차 회의221.일자2009년 월 일 금)109(...
